In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [7]:
# parameter grid
distances = [3, 11] #, 40, 45, 50, 55]
rounds = [3, 11]
devices = ["ibmq_mumbai"]#, "ibm_brisbane", "ibm_nazca", "ibm_cusco"]
logicals = ['0', '1']
print(distances[::-1])

[11, 3]


In [9]:
from tqdm import tqdm

from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit import transpile

from soft_info import get_repcode_layout, find_longest_path_in_hex, find_longest_path_general
from Scratch import metadata_helper



for distance in tqdm(distances):  
    for round in rounds:
        code = RepetitionCodeCircuit(distance, round) 
        for device in devices:
            backend = provider.get_backend(device)
            layout = get_repcode_layout(distance, backend, _is_hex=False)
            for logical in logicals:          
                qc = code.circuit[logical]                                   
                transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=3)

                n_shots = int(1e6 / (distance * round))
                metadata = metadata_helper(descr="Run bigger Repetition codes v4: new distances faster triple loop" ,code="RepetitionCodeCircuit", distance=distance, rounds=distance, logical=logical, layout='_is_hex=True')
                backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=0, meas_return='single', job_tags=['RepCode_Count_job_count']) # meas_level=1 for IQ  
                


100%|██████████| 2/2 [00:39<00:00, 19.98s/it]
